# Advanced Lane Finding Project

The goals / steps of this project are the following:

- [x] Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
- [x] Apply a distortion correction to raw images.
- [x] Use color transforms, gradients, etc., to create a thresholded binary image.
- [x] Apply a perspective transform to rectify binary image ("birds-eye view").
- [ ] Detect lane pixels and fit to find the lane boundary.
- [ ] Determine the curvature of the lane and vehicle position with respect to center.
- [ ] Warp the detected lane boundaries back onto the original image.
- [ ] Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

## Imports

In [ ]:
# builtins
from glob import glob
import functools
import os

# third-party
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# project-specific
from src.calibrate import calibrate
from src.thresholding import region_of_interest, thresh, and_binary, or_binary
from src.image_mappings import direction, magnitude
from src.perspective import get_transformers

## Calibration

In [ ]:
print("Finding all calibration images...")
calibration_image_paths = sorted(glob('./camera_cal/calibration*.jpg'), key=lambda p: (len(p), p))
calibration_images = list(map(cv2.imread, calibration_image_paths))
for path, image in zip(calibration_image_paths, calibration_images):
    print("{0} ({1[0]}w, {1[1]}h)".format(
        os.path.basename(path),
        image.shape
    ))
print("Found {} images.".format(len(calibration_images)))

print("Calibrating...")
used_calibration_images, object_points, all_corners, undistort, mtx, dist = calibrate(
    calibration_images, 
    chessboard_dimensions=(9, 6), 
    to_grayscale_flag=cv2.COLOR_BGR2GRAY
)
print("Calibrated using {} chessboard images (was unable to use {} images).".format(
    len(used_calibration_images), len(calibration_images) - len(used_calibration_images)
))

print("Writing to output_images/chessboards...")
directory = "output_images/chessboards"
try:
    os.mkdir(directory)
except:
    pass

for image, corners, i in zip(used_calibration_images, all_corners, range(len(used_calibration_images))):
    img = cv2.drawChessboardCorners(image, (9, 6), corners, True)
    cv2.imwrite(directory + "/{}.before.jpg".format(i), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    out = undistort(img)
    cv2.imwrite(directory + "/{}.after.jpg".format(i), cv2.cvtColor(out, cv2.COLOR_RGB2BGR))
    if i == 0:
        plt.imshow(out)

print("Written. Sample output with corners drawn:")

## Perspective Transform

In [ ]:
to_birds_eye, to_car_perspective = get_transformers(np.float32([
    [600, 450],
    [683, 450],
    [1060, 690],
    [253, 690],
]), np.float32([
    [350, 0],
    [950, 0],
    [950, 720],
    [350, 720],
]))

## Thresholding

In [ ]:
print("Defining image selectors...")
# Yes. Lots of fiddling with numbers happened here. Please, never again.

def asphalt_selector(img, convert_to_hls_flag=cv2.COLOR_BGR2HLS, convert_to_gray_flag=cv2.COLOR_BGR2GRAY):
    hls = cv2.cvtColor(img, convert_to_hls_flag)
    gray = cv2.cvtColor(img, convert_to_gray_flag)
    return and_binary(
        thresh(direction(
            thresh(
                hls[:,:,1], 
                (140, 240)
            ), 
            sobel_kernel=5
        ), (0.7, 1.8)),
        # &
        thresh(magnitude(
            gray,
            sobel_kernel=13
        ), (100, 256)),
    )

def concrete_selector(img, convert_to_hls_flag=cv2.COLOR_BGR2HLS):
    hls = cv2.cvtColor(img, convert_to_hls_flag)
    return thresh(magnitude(
            thresh(hls[:,:,2], (200, 256)),
            sobel_kernel=13,
        ), (110, 256),
    )

def region_selector(i):
    return region_of_interest(
        i, 
        np.array([[
            ((i.shape[1] * 1) / 16, i.shape[0]),
            ((i.shape[1] * 7) / 16, i.shape[0] * .61),
            ((i.shape[1] * 9) / 16, i.shape[0] * .61),
            ((i.shape[1] * 15) / 16, i.shape[0])
        ]], dtype=np.int32),
    )


def lane_selector(img, convert_to_hls_flag=cv2.COLOR_BGR2HLS, convert_to_gray_flag=cv2.COLOR_BGR2GRAY):
    return region_selector(
        or_binary(
            asphalt_selector(img, convert_to_hls_flag=convert_to_hls_flag, convert_to_gray_flag=convert_to_gray_flag),
            concrete_selector(img, convert_to_hls_flag=convert_to_hls_flag),
        )
    )

print("Defined.")


In [ ]:
print("Showing samples using the selectors...")
img = cv2.imread("test_images/test1.jpg")
perp = to_birds_eye(img)
hls = cv2.cvtColor(perp, cv2.COLOR_BGR2HLS)

f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(24, 9))
f.tight_layout()
ax1.imshow(to_birds_eye(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)))
ax2.imshow(to_birds_eye(asphalt_selector(img)), cmap="gray")
ax3.imshow(to_birds_eye(concrete_selector(img)), cmap="gray")
ax4.imshow(to_birds_eye(region_selector(lane_selector(img))), cmap="gray")
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
print("Done.")

## Detect Lane Pixels

In [ ]:
print("Defining lane pixel detectors...")

def get_lane_start(img):
    histogram = np.sum(img[img.shape[0]//2:,:], axis=0)
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    return (leftx_base, rightx_base)

def get_lane_pixels(img, nwindows=9, margin=100, minpix=50):
    window_height = np.int(img.shape[0] // nwindows)
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # to debug:
    # out_img = np.dstack((img, img, img))

    lane_inds = [
        [],  # left lane
        [],  # right lane
    ]
    current = list(get_lane_start(img))  # leftx_current, rightx_current

    for window in range(nwindows):
        window_boundaries = list(
            map(lambda b: (b - margin, b + margin), current)
        )
        win_y_low = img.shape[0] - (window + 1) * window_height
        win_y_high = img.shape[0] - window * window_height
        
        for i, bound in enumerate(window_boundaries):
            leftx, rightx = bound
            
            # to debug:
            # cv2.rectangle(out_img, (leftx, win_y_low), (rightx, win_y_high), (0, 255, 0), 2) 

            good_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= leftx) &  (nonzerox < rightx)).nonzero()[0]
            
            lane_inds[i].append(good_inds)
            
            if len(good_inds) > minpix:
                current[i] = np.int(np.mean(nonzerox[good_inds]))

    # to debug:
    # plt.imshow(out_img)
    lane_inds = list(map(np.concatenate, lane_inds))
    return [(nonzerox[inds], nonzeroy[inds]) for inds in lane_inds ]


def fit_polynomials(img):
    ploty = np.linspace(0, img.shape[0] - 1, img.shape[0])    
    lane_pixels = get_lane_pixels(img)
    quads = list(map(
        lambda vp: np.polyfit(vp[1], vp[0], 2),
        lane_pixels,
    ))
    return list(map(
        lambda fit: fit[0] * (ploty ** 2)
            + fit[1] * (ploty ** 1)
            + fit[2], 
        quads,
    ))

print("Done.")

img = to_birds_eye(lane_selector(cv2.imread("test_images/test1.jpg")))
left_fit, right_fit = fit_polynomials(img)
ploty = np.linspace(0, img.shape[0] - 1, img.shape[0])
plt.plot(left_fit, ploty, color='yellow')
plt.plot(right_fit, ploty, color='yellow')
plt.imshow(img)

In [ ]:
print("Processing sample images...")

for image_name in sorted(glob("test_images/*.jpg"), key=lambda p: (len(p), p)):
    img = cv2.imread(image_name)
    directory = "output_images/" + os.path.basename(image_name)
    try:
        os.mkdir(directory)
    except:
        pass
    print(directory)

    cv2.imwrite(directory + "/original.jpg", img)
    
    # Undistort image (virtue of camera lens)
    undis = undistort(img)
    cv2.imwrite(directory + "/undistorted.jpg", undis)
    
    # Select lanes
    asphalt = to_birds_eye(asphalt_selector(undis))  # for show
    plt.imsave(directory + "/asphalt_selector.jpg", asphalt, cmap='gray')
    concrete = to_birds_eye(concrete_selector(undis))  # for show
    plt.imsave(directory + "/concrete_selector.jpg", concrete, cmap='gray')

    lanes = to_birds_eye(lane_selector(undis))
    plt.imsave(directory + "/lane_selector.jpg", lanes, cmap='gray')

print("Done.")
